In [ ]:
# delete all variables
%reset

In [ ]:
# load necessary modules
from __future__ import print_function

import os
import re
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import SVC
import pickle
from itertools import combinations
%matplotlib inline

In [ ]:
# get the action units from folders
# The 3 folders contain action units extracted by a pretrained network for each image
files_train = os.listdir('../../data/action_units/pretrained/Train/')
files_valid = os.listdir('../pretrained/Valid/')
files_test = os.listdir('../pretrained/Test/')

train_csv = [file for file in files_train if 'csv' in file]
valid_csv = [file for file in files_valid if 'csv' in file]
test_csv = [file for file in files_test if 'csv' in file]

# Get the indices for every set
ind_train = sorted([int(re.search('\d+', tr).group()) for tr in train_csv])
ind_valid = sorted([int(re.search('\d+', v).group()) for v in valid_csv])
ind_test = sorted([int(re.search('\d+', te).group()) for te in test_csv])

In [ ]:
# Get the emotion from every image
def get_emotion(index, folder):
    tmp = np.load(folder + 'lab{}.npy'.format(index))
    m = np.max(tmp)
    indexes = [i for i, j in enumerate(tmp) if j == m]
    if len(indexes) == 1:
        return indexes[0]
    else:
        return None

In [ ]:
# Create a dataset for all the AU for the training set
df_train = pd.DataFrame()
for i in tqdm(ind_train):
    em = get_emotion(i, '../../data/action_units/pretrained/Train/')
    if em is not None:
        f = pd.read_csv('../../data/action_units/pretrained/Train/img{}.csv'.format(i))
        f['emotion'] = int(em)
        df_train = pd.concat([df_train,f])

In [ ]:
# Create a dataset for all the AU for the validation set
df_valid = pd.DataFrame()
for i in tqdm(ind_valid):
    em = get_emotion(i, '../../data/action_units/pretrained/Valid/')
    if em is not None:
        f = pd.read_csv('../../data/action_units/pretrained/Valid/img{}.csv'.format(i))
        f['emotion'] = int(em)
        df_valid = pd.concat([df_valid,f])

In [ ]:
# Create a dataset for all the AU for the test set
df_test = pd.DataFrame()
for i in tqdm(ind_test):
    em = get_emotion(i, '../../data/action_units/pretrained/Test/')
    if em is not None:
        f = pd.read_csv('../../data/action_units/pretrained/Test/img{}.csv'.format(i))
        f['emotion'] = int(em)
        df_test = pd.concat([df_test,f])

In [ ]:
df_train = pd.concat([df_train, df_valid])

In [ ]:
# Save the dataframes to a csv file
df_train.to_csv('./../data/action_units/au_train.csv', index=False)
df_test.to_csv('./../data/action_units/au_test.csv', index=False)